# `anato-mesh` in Jupyter Notebook 

`anato_mesh.py` contains the main functions for calculating the partition-level curvatures using python to reproduce the algorithm originally published by K. Khabaz here: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011815

In [ ]:
from anato_mesh import *
from anato_viz import *
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [ ]:
#- Pocivavsek lab parent directory
parent_path = 'Z:\\aorta\\thoracic'
#- Cohort filter
group_str = ['KK']
#- Mesh filter
file_str = ['M5']

<table>
    <tr>
        <th style="text-align:center">quantities</th>
        <th style="text-align:center">Equation</th>
    </tr>
    <tr>
        <td style="text-align:center">'Gaussian'</td>
        <td style="text-align:center">$$\displaystyle k_1 \cdot k_2$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'Mean'</td>
        <td style="text-align:center">$$\displaystyle 0.5 \cdot (k_1 + k_2)$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'IntGaussian'</td>
        <td style="text-align:center">$$\displaystyle (k_1 \cdot k_2) \cdot A$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'IntMeanSquared'</td>
        <td style="text-align:center">$$\displaystyle \left(0.5 \cdot (k_1 + k_2)\right)^2 \cdot A$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'Willmore'</td>
        <td style="text-align:center">$$\displaystyle 4 \cdot \left(0.5 \cdot (k_1 + k_2)\right)^2 \cdot A - 2 \cdot (k_1 \cdot k_2) \cdot A$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'Casorati'</td>
        <td style="text-align:center">$$\displaystyle \sqrt{0.5 \cdot (k_1^2 + k_2^2)}$$</td>
    </tr>
    <tr>
        <td style="text-align:center">'ShapeIndex'</td>
        <td style="text-align:center">$$\displaystyle \frac{2}{\pi} \cdot \arctan\left(\frac{k_2 + k_1}{k_2 - k_1}\right)$$</td>
    </tr>
</table>


You must specify at least one variable in `quantitites` and must also define the `point_removal` method. When in doubt, use the `curvature` method because it is applicable to all geometries. The `thoracic` specific point removal process is still buggy, overly empirical, and is being fixed. If the mesh is ready to be processed as is, then set `None`.

In [ ]:
quantities = ['Gaussian', 'Mean', 'IntGaussian', 'IntMeanSquared', 'Willmore', 'Casorati', 'ShapeIndex']
point_removal = 'curvature'

The number of surface partitions for each surface is determined by the following equation. `m_set` provides an easily adjustable parameter for further partition scaling and allows you to calculate more than one scaling at once. The optimized value for the thoracic aorta TEVAR dataset is `m=1`.

$$partitions = m \times \left(\frac{SA}{R^2}\right)$$

In [ ]:
m_set = [1]

#### Executing the batch run: 

In [ ]:
results, scan_dict = GetAnatoMeshResults(parent_path, group_str, file_str, point_removal, quantities, m_set)

In [ ]:
scan_dict

### If you just want data, stop here. 
#### Everything after this are postprocessing steps including normalization, meta-data integration, visualization, and exporting.

Normalizing the results. This works by first splitting up the compound name using the `SplitUpScanName` function and then normalizing the variables in the 2nd and 3rd function inputs to the `GroupsNormalize` function by the group of datapoints that contain the string sequence in the 4th function input. This is normally `KY` patients for TEVAR and `JX` patients for EVAR in our data. 

In [ ]:
results_norm = GroupsNormalize(SplitUpScanName(results), 'Casorati_Mean', 'IntGaussian_Fluct', 'JX')

Merging meta data files into the anato-mesh results.

In [ ]:
#- Meta data integration 
directory = 'Z:\\aorta\\meta_data'
file_name = 'MasterMetaData.xlsx'

#- Cohort meta data group names
cohort_list = ['UC_NORMAL', 'UC_PEDS', 'UC_TEVAR', 'ENDOSPAN', 'MEDTRONIC', 'GORE_801', 'GORE_802', 'GORE_803']

#- Integer columns to be converted to strings for discrete visualization
cat_columns = ['Label', 'Outcome']

In [ ]:
results_meta = MergeMetaData(directory, file_name, cohort_list, results_norm, cat_columns)

Ploting and saving the data. 

In [ ]:
fig = px.scatter(results_norm, x='Mean_Radius', y='Total_Fluct_Norm')#, color='Label', hover_data=['Scan_Name'])
fig.show()

In [ ]:
SaveResultsToXLSX('path', 'data.xlsx', results_meta)

In [ ]:
def SaveScanDictToXLSX(directory, file_name, scan_dict):
    """Save a dictionary as an .xlsx file."""
    Path(directory).mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(Path(directory) / file_name) as writer:
        for main_key, sub_dict in scan_dict.items():
            data = []
            for key, values, in sub_dict.items():
                df = pd.DataFrame(values)
                df['key'] = key[:-3]
                data.append(df)
            output_df = pd.concat(data, ignore_index=True)
            output_df.to_excel(writer, sheet_name=str(main_key), index=False)

In [ ]:
SaveScanDictToXLSX(r'Z:\aorta\graphs', 'KK_floating_patches.xlsx', scan_dict) 